### 使用流程
1. 建立Insert、Clear等各項function
2. 建立資料庫、表格

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import warnings
import sqlite3  
output_path='./output/cut/'
db_name = './data/sqlchain.db'
conn = sqlite3.connect(db_name)  

In [3]:
# 基本設定
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format',lambda x: '%.2f' %x)
warnings.filterwarnings("ignore")

In [4]:
# Insert Data
def insertTable(insert_table, records_table):
    # 直接將DataFrame轉換成元组列表
    records = [tuple(x) for x in records_table.values]

    conn = sqlite3.connect(db_name)
    try:
        # 假設DataFrame的列名與資料表的列名一致
        columns = ",".join(records_table.columns)
        placeholders = ", ".join(["?"] * len(records_table.columns))
        insert_data_sql = f"INSERT INTO {insert_table} ({columns}) VALUES ({placeholders})"
        
        conn.executemany(insert_data_sql, records)
        conn.commit()
    except sqlite3.Error as error:
        print(error)
    finally:
        conn.close()


In [5]:
# Clear Data
def clearTable(db_name, table_name):
    """
    清空指定SQLite資料表中的所有數據。

    参数:
    db_name (str): 資料表文件名。
    table_name (str): 要清空的表名。
    """
    try:
        conn = sqlite3.connect(db_name)
        conn.execute(f'''DELETE FROM {table_name};''')
        conn.commit()
    except sqlite3.Error as error:
        print(f"error: {error}")
    finally:
        conn.close()


### Insert CPU 的資料

In [13]:
table_name='cpu'
clearTable(db_name, table_name)

In [14]:
#讀取資料
df_cpu=pd.read_csv(output_path+'cpu_cut_v2.csv')
df_cpu.drop(columns=['name','class','model'],inplace=True)
df_cpu.rename(columns={'cut_name':'name'},inplace=True)
#插入資料
insertTable('cpu', df_cpu)

### Insert GPU 的資料

In [104]:
# 清空資料(這邊是為了避免資料因為爬蟲出現重複，所以先清空table全部資料)
table_name='gpu'
clearTable(db_name, table_name)

In [105]:
#清理一下Brand的資料(只包含Nvidia、AMD)
def check_and_set_brand(row):
    # 檢查 MODEL 是否包含特定字眼，並相應設置 BRAND
    if any(keyword in row['model'] for keyword in ['GT', 'NVIDIA', 'RTX']):
        return 'Nvidia'
    elif row['model'].startswith('RX'):
        return 'AMD'
    else:
        return row['brand']  # 保留原有值



In [106]:
#讀取資料
df_gpu=pd.read_csv(output_path+'gpu_cut.csv')
print(df_gpu.shape)
df_gpu.drop(columns=['class','cudacores','name'],inplace=True)
df_gpu['brand'] = df_gpu.apply(check_and_set_brand, axis=1)
df_gpu.rename(columns={'cut_name':'name'},inplace=True)
df_gpu=df_gpu.loc[(df_gpu['brand']=='Nvidia')|(df_gpu['brand']=='AMD')]
print(df_gpu.shape)
df_gpu.head()

(270, 14)
(253, 11)


,price,etl_date,model,brand,memory,memory_value,memory_unit,corefrequency_value,corefrequency_unit,ddr,name
1,1690,2024-03-17,GT710,Nvidia,2G,2.00,G,954.00,MHz,3.00,華碩 GT710-SL-2GD3-BRK-EVO
2,1790,2024-03-17,GT710,Nvidia,2G,2.00,G,954.00,MHz,5.00,華碩 GT710-SL-2GD5-BRK-EVO
4,1450,2024-03-17,GT710,Nvidia,1G,1.00,G,954.00,MHz,3.00,微星 GT710 1GD3H LP
5,1790,2024-03-17,GT710,Nvidia,2G,2.00,G,954.00,MHz,3.00,微星 GT710 2GD3H LP
6,1388,2024-03-17,GT710,Nvidia,1GB,1.00,GB,954.00,MHz,3.00,INNO3D GT710 1GB DDR3


In [107]:
#插入資料
insertTable('gpu', df_gpu)

# Insert 硬碟的資料

In [62]:
table_name='hdd'
clearTable(db_name, table_name)

In [63]:
df_hdd=pd.read_csv(output_path+'hdd_cut.csv')
df_hdd.drop(columns=['id','class','capacity','model','name'],inplace=True)
df_hdd.rename(columns={'cut_name':'name'},inplace=True)
#插入資料
insertTable('hdd', df_hdd)

## Insert Ram的資料

In [29]:
table_name='ram'
clearTable(db_name, table_name)

In [30]:
df_ram=pd.read_csv(output_path+'ram_cut.csv')
#刪除容量是負的贓資料
print(df_ram.shape)
df_ram=df_ram.loc[df_ram['capacity_value']>0]
df_ram.drop(columns=['class','name'],inplace=True)
df_ram.rename(columns={'cut_name':'name'},inplace=True)
print(df_ram.shape)
#插入資料
insertTable('ram', df_ram)

(308, 10)
(303, 8)


# Insert 散熱器的資料

In [57]:
table_name='radiator'
clearTable(db_name, table_name)

In [58]:
df_radiator=pd.read_csv(output_path+'cool_cut.csv')
df_radiator.drop(columns=['class','model','name'],inplace=True)
df_radiator.rename(columns={'cut_name':'name'},inplace=True)
#插入資料
insertTable('radiator', df_radiator)

## 處理機殼的資料

In [59]:
table_name='chassis'
clearTable(db_name, table_name)

In [60]:
df_chassis=pd.read_csv(output_path+'chassis_cut.csv')
df_chassis.drop(columns=['class','name'],inplace=True)
df_chassis.rename(columns={'cut_name':'name'},inplace=True)
#插入資料
insertTable('chassis', df_chassis)

## 處理電源的資料 

In [175]:
table_name='battery'
clearTable(db_name, table_name)

In [80]:
df_battery.head()

,price,etl_date,tdp,cooling_method,model,name
0,4990,2024-03-03,850,NaN,NaN,華碩 ROG STRIX 850W AURA Edition
1,2590,2024-03-03,750,NaN,NaN,美洲獅 COUGAR GEX 750W
2,3990,2024-03-03,1050,NaN,NaN,美洲獅 COUGAR GEX 1050W
3,2390,2024-03-03,750,NaN,NaN,銀欣 750W
4,4990,2024-03-03,1100,NaN,NaN,台達 超實在 1100W


In [77]:
df_battery=pd.read_csv(output_path+'battery_cut.csv')
df_battery.drop(columns=['class','name','tdp','model'],inplace=True)
df_battery.rename(columns={'cut_name':'name'},inplace=True)
#插入資料
insertTable('battery', df_chassis)

table battery has no column named brand


# 處理主機板的資料

In [20]:
table_name='mainboard'
clearTable(db_name, table_name)

In [21]:
df_mainboard=pd.read_csv(output_path+'motherboard_cut.csv')
df_mainboard.drop(columns=['class','name','model','size','company'],inplace=True)
df_mainboard.rename(columns={'cut_name':'name'},inplace=True)
insertTable('mainboard', df_mainboard)
df_mainboard.head()

,price,etl_date,brand,name
0,399,2024-03-03,華碩,華碩 Tinker Fanless Aluminum Case
1,12990,2024-03-03,華碩,華碩 PRO WS W680M-ACE SE
2,9990,2024-03-03,華碩,華碩 PRO WS W680-ACE
3,39990,2024-03-03,華碩,華碩 Pro WS W790E-SAGE SE
4,34990,2024-03-03,Unknown,華擎 W790 WS


In [ ]:
df_mainboard=pd.read_csv(output_path+'monitor_cut.csv')
df_monitor.drop(columns=['class','name','model'],inplace=True)
df_monitor.rename(columns={'cut_name':'name'},inplace=True)
#插入資料
insertTable('monitor', df_monitor)

### 處理螢幕的資料

In [8]:
df_monitor=pd.read_csv(output_path+'monitor_cut.csv')
df_monitor.head()

,class,name,price,etl_date,model,size,brand,company,cut_name
0,主機板 MB,華碩 Tinker Fanless Aluminum Case 樹莓派鋁質機殼 (2/2S ...,399,2024-03-03,Unknown,Unknown,華碩,華碩,華碩 Tinker Fanless Aluminum Case
1,主機板 MB,華碩 PRO WS W680M-ACE SE(M-ATX/1A1H1P/雙Intel 2.5...,12990,2024-03-03,M-ACE,ATX,華碩,華碩,華碩 PRO WS W680M-ACE SE
2,主機板 MB,華碩 PRO WS W680-ACE(ATX/1A1H1P/雙Intel 2.5Gb/註四年...,9990,2024-03-03,Unknown,ATX,華碩,華碩,華碩 PRO WS W680-ACE
3,主機板 MB,華碩 Pro WS W790E-SAGE SE(EEB/8*DDR5/2*Intel 10G...,39990,2024-03-03,E-SAGE,Unknown,華碩,華碩,華碩 Pro WS W790E-SAGE SE
4,主機板 MB,華擎 W790 WS(EEB/8*DDR5/2*Marvell 10G+Intel 2.5G...,34990,2024-03-03,Unknown,Unknown,Unknown,Unknown,華擎 W790 WS


In [74]:
table_name='monitor'
clearTable(db_name, table_name)

In [75]:
df_monitor=pd.read_csv(output_path+'monitor_cut.csv')
df_monitor.drop(columns=['class','name','model'],inplace=True)
df_monitor.rename(columns={'cut_name':'name'},inplace=True)
#插入資料
insertTable('monitor', df_monitor)

## 驗證查詢

In [27]:
### 驗證查詢
conn = sqlite3.connect(db_name)  
sql1=f'''
select *
from ram
order by capacity_value asc
'''
test=pd.read_sql(sql1,conn)
conn.close()
test.head()

,NAME,PRICE,BRAND,ETL_DATE,module_Spec,capacity,capacity_value,capacity_unit
0,美光Micron Crucial NB 雙通16G*2 DDR5 4800/CL40,2699,美光Micron,2024-03-03,DDR5,Unknown,-1,Unknown
1,美光Micron Crucial NB 雙通32G*2 DDR5 4800/CL40,5399,美光Micron,2024-03-03,DDR5,Unknown,-1,Unknown
2,美光Micron Crucial NB 雙通16G*2 DDR5 5600/CL46,2799,美光Micron,2024-03-03,DDR5,Unknown,-1,Unknown
3,美光Micron Crucial NB 雙通32G*2 DDR5 5600/CL46,5599,美光Micron,2024-03-03,DDR5,Unknown,-1,Unknown
4,金士頓 8G DDR3-1600,1150,金士頓,2024-03-03,DDR3,Unknown,-1,Unknown
